In [1]:
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif, f_classif, SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import random

import pandas as pd
import os 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import datetime as dt
from datetime import datetime

%matplotlib inline

In [3]:
def convert_unix_time(ts):
    if np.isnan(ts):
        return datetime.strptime('9999-12-31 23:59:59' , '%Y-%m-%d %H:%M:%S')
    ts = int(ts)
    return datetime.strptime(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'),'%Y-%m-%d %H:%M:%S')

In [46]:
print(dt.datetime.now())
# there is one dup in df_users.
df_users = pd.read_csv('takehome_users.csv',encoding='latin1' ).drop_duplicates()

# drop records with null last_creation_time
ilist = df_users.loc[:,'last_session_creation_time'].isnull()
drop_idx = df_users.loc[list(ilist),'last_session_creation_time'].index
df_users.drop(drop_idx, axis=0, inplace=True)
print(df_users.shape)

df_users['last_session_creation_datetime'] = df_users.last_session_creation_time.apply(convert_unix_time)
df_users['creation_time'] = [datetime.strptime(dt, '%Y-%m-%d %H:%M:%S') for dt in df_users.creation_time.values]
df_user_engagement = pd.read_csv('takehome_user_engagement.csv',encoding='latin1' )
df_user_engagement['txn_dt'] = [datetime.date(datetime.strptime(dt.split(' ')[0] , '%Y-%m-%d')) \
                                for dt in df_user_engagement['time_stamp']]

print(dt.datetime.now())
df_users['adopted'] = 0
df_users['three_engagements'] = 0
ulist = []
t = df_user_engagement.groupby(['user_id'])['visited'].count()
ul = [u for u in t.index if t[u] > 2]
print(df_users.adopted.sum())
print(len(ul))
df_users.loc[df_users['object_id'].isin(ul),'three_engagements'] = 1
for u in ul:
    for i in df_user_engagement.loc[df_user_engagement.user_id == u, :].index:
        day_one = df_user_engagement.loc[i, 'txn_dt']
        d1 = (df_user_engagement.loc[i+1, 'txn_dt'] - day_one).days
        d2 = (df_user_engagement.loc[i+2, 'txn_dt'] - day_one).days
        if d1 < 7 and d2 < 7:
            ulist.append(u)
            break

df_users.loc[df_users.object_id.isin(ulist), 'adopted'] = 1
print(df_users.adopted.sum())
print(df_users.three_engagements.sum())
print(dt.datetime.now())

2019-04-21 09:51:07.929861
(8823, 10)
2019-04-21 09:51:12.192513
0
2248
1959
2248
2019-04-21 09:51:15.103205


In [101]:
df_user_engagement = df_user_engagement.set_index('time_stamp')
df_user_engagement.head()

,user_id,visited,txn_dt
time_stamp,,,
2014-04-22 03:53:30,1,1,2014-04-22
2013-11-15 03:45:04,2,1,2013-11-15
2013-11-29 03:45:04,2,1,2013-11-29
2013-12-09 03:45:04,2,1,2013-12-09
2013-12-25 03:45:04,2,1,2013-12-25


In [104]:
df_user_engagement.rolling('7D')['object_id'].count() 

ValueError: window must be an integer

In [76]:
sum(df_users.invited_by_user_id.isnull())

4047

In [84]:
null_idx = df_users.loc[df_users.invited_by_user_id.isnull(),:]
len(null_idx)

0

In [85]:
df_users.invited_by_user_id.fillna(0, inplace=True)

In [86]:
df_users.loc[:,'invited_by_user_id'].value_counts()

0.0        4047
4612.0       10
2308.0        9
11770.0       9
1525.0        8
2354.0        8
4803.0        8
730.0         8
6706.0        8
6808.0        8
4302.0        8
2527.0        8
3261.0        8
10628.0       8
3721.0        8
10741.0       8
8043.0        7
8026.0        7
10127.0       7
5775.0        7
3641.0        7
7928.0        7
7170.0        7
9434.0        7
9223.0        7
1656.0        7
1846.0        7
4908.0        7
4118.0        7
10481.0       7
           ... 
4077.0        1
3191.0        1
10869.0       1
4727.0        1
6759.0        1
5749.0        1
2302.0        1
1940.0        1
7028.0        1
3480.0        1
4187.0        1
8202.0        1
10823.0       1
10030.0       1
153.0         1
10334.0       1
11961.0       1
11848.0       1
743.0         1
8847.0        1
3689.0        1
3188.0        1
331.0         1
6682.0        1
3583.0        1
3337.0        1
10797.0       1
5139.0        1
9819.0        1
10941.0       1
Name: invited_by_user_id

In [78]:
ilist = df_users.loc[:,'invited_by_user_id'].isnull()
ilist = list(ilist)
print(len(ilist))
df_users.loc[ilist,:].head()
#df_users.drop(drop_idx, axis=0, inplace=True)
#df_users.shape

8823


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,last_session_creation_datetime,adopted,three_engagements
6,7,2012-12-16 13:24:32,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN,2012-12-20 13:24:32,0,0
10,11,2013-12-26 03:55:54,Paulsen Malthe,MaltheAPaulsen@gustr.com,SIGNUP,1.388117e+09,0,0,69,NaN,2013-12-27 03:55:54,0,0
13,14,2012-10-11 16:14:33,Rivera Bret,BretKRivera@gmail.com,SIGNUP,1.350058e+09,0,0,0,NaN,2012-10-12 16:14:33,0,0
18,19,2013-05-24 14:56:36,Collins Arlene,ArleneRCollins@gmail.com,SIGNUP,1.369926e+09,1,0,0,NaN,2013-05-30 14:56:36,0,0
19,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN,2014-05-29 11:46:38,1,1


In [ ]:
n=0
ilist = []
for i in df_users.index:
    #print(i)
    try:
        d = (df_users.loc[i,'last_session_creation_datetime'] - df_users.loc[i,'creation_time']).days
    except:
        print('error')
        print(df_users.loc[i,'last_session_creation_datetime'])
        print(df_users.loc[i,'creation_time'])
        print('='*20)
        n+=1
        ilist.append(i)
print(n)

In [22]:
df_users.loc[df_users.index.isin(ilist), :][:5]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,last_session_creation_datetime,adopted,three_engagements
7,8,2013-07-31 05:34:02,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN,9999-12-31 23:59:59,0,0
8,9,2013-11-05 04:04:24,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN,9999-12-31 23:59:59,0,0
11,12,2014-04-17 23:48:38,Mathiesen Lærke,LaerkeLMathiesen@cuvox.de,ORG_INVITE,NaN,0,0,130,9270.0,9999-12-31 23:59:59,0,0
14,15,2013-07-16 21:33:54,Theiss Ralf,RalfTheiss@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,175,NaN,9999-12-31 23:59:59,0,0
15,16,2013-02-11 10:09:50,Engel René,ReneEngel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,211,NaN,9999-12-31 23:59:59,0,0


In [25]:
df_users.loc[df_users.index.isin(ilist), 'creation_source'].value_counts()

PERSONAL_PROJECTS    1347
ORG_INVITE           1066
GUEST_INVITE          575
SIGNUP                189
Name: creation_source, dtype: int64

In [21]:
df_users.loc[df_users.index.isin(ilist), ['object_id','creation_time','last_session_creation_time',\
                                          'last_session_creation_datetime']][:5]

,object_id,creation_time,last_session_creation_time,last_session_creation_datetime
7,8,2013-07-31 05:34:02,NaN,9999-12-31 23:59:59
8,9,2013-11-05 04:04:24,NaN,9999-12-31 23:59:59
11,12,2014-04-17 23:48:38,NaN,9999-12-31 23:59:59
14,15,2013-07-16 21:33:54,NaN,9999-12-31 23:59:59
15,16,2013-02-11 10:09:50,NaN,9999-12-31 23:59:59


In [ ]:
print(df_users.creation_time.head())
ts = df_users.last_session_creation_time.head()
ts = ts.values.astype('int')
print(ts)

for i in ts:
    print(datetime.utcfromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
print(df_user_engagement.shape)
df_user_engagement.visited.sum()

In [87]:
df_user = pd.merge(df_users, pd.get_dummies(df_users.creation_source), left_index=True, right_index=True)
df_user = df_user.drop(columns=['creation_time','name','email','creation_source',\
                                'last_session_creation_time'])
df_user.head()

,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,last_session_creation_datetime,adopted,three_engagements,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1,1,0,11,10803.0,2014-04-22 03:53:30,0,0,1,0,0,0,0
1,2,0,0,1,316.0,2014-03-31 03:45:04,1,1,0,1,0,0,0
2,3,0,0,94,1525.0,2013-03-19 23:14:52,0,0,0,1,0,0,0
3,4,0,0,1,5151.0,2013-05-22 08:09:28,0,0,1,0,0,0,0
4,5,0,0,193,5240.0,2013-01-22 10:14:20,0,0,1,0,0,0,0


In [88]:
ulist = []
usrs = list(set(df_user_engagement.user_id))
len(usrs)
df_user = df_user.loc[df_user.object_id.isin(usrs),:]
df_user.shape
#for u in df_user.object_id.values:
    

(8823, 13)

In [89]:
idx = df_user.loc[df_user.three_engagements == 0, :].index
idx1 = df_user.loc[df_user.three_engagements == 1, :].index
print(len(idx1))
len(idx)

2248


6575

In [90]:

def get_vals():
    """ A function to balance out the data. There are many more users that were not considered to be 'adopted users.'
    So a model that predicts 'not an adopted user' would be correct about 80% of the time.
    """
    num = 2248
    
    idx = df_user.loc[df_user.three_engagements == 0, :].index
    idx1 = df_user.loc[df_user.three_engagements == 1, :].index
    sampling = random.choices(idx, k=num)
    
    return sampling + list(idx1)
len(get_vals())

4496

In [110]:
#df_user = df_user.loc[get_vals(),:]
print(df_user.shape)
drop_list = ['adopted','object_id', 'last_session_creation_datetime']

# split the data up into test and train data sets.
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_user.drop(columns=drop_list), \
                                                                df_user.adopted, test_size=0.3)
print(X_test.shape)
print(y_train.shape)
X_train.shape

(4496, 13)
(1349, 10)
(3147,)


(3147, 10)

In [111]:
X_train.columns

Index(['opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id',
       'invited_by_user_id', 'three_engagements', 'GUEST_INVITE', 'ORG_INVITE',
       'PERSONAL_PROJECTS', 'SIGNUP', 'SIGNUP_GOOGLE_AUTH'],
      dtype='object')

In [112]:
# Logistic regression
print (dt.datetime.now())

# create a pipeline with a Logistic Regression classifier as the model. Next, use cross validation to ensure the
# model is not overfitting or underfitting.

pipe = Pipeline([
       #('scaler', StandardScaler()),
       ('clf', LogisticRegression()) 
    ])


cv = model_selection.ShuffleSplit(n_splits=5, test_size=0.2)
param_grid = { "clf__solver": ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'] } 
pipe_est_cv = model_selection.GridSearchCV(pipe, param_grid=param_grid, cv=cv, \
                                           scoring='accuracy', verbose=3, n_jobs=2)

pipe_est_cv.fit(X_train, y_train)
pipe_est_cv_results = pipe_est_cv.cv_results_

print(pipe_est_cv.score(X_train, y_train))
print(pipe_est_cv.score(X_test,y_test))

print(pipe_est_cv.best_params_)
print(pipe_est_cv.best_score_)


print(pipe_est_cv_results['mean_test_score'])
print (dt.datetime.now())

2019-04-21 10:56:04.272053
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:   17.2s finished


0.9313632030505243
0.9458858413639734
{'clf__solver': 'newton-cg'}
0.9225396825396825
[0.92253968 0.92253968 0.92253968 0.57238095 0.57111111]
2019-04-21 10:56:21.899629


/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


In [113]:
from sklearn.metrics import classification_report

# precision = true positives / (true positives + false positives)
# recall    = true positives / (true positives + false negatives)

y_pred = pipe_est_cv.predict(X_test)
metrics = classification_report(y_test, y_pred)
print(metrics)

              precision    recall  f1-score   support

           0       1.00      0.90      0.95       746
           1       0.89      1.00      0.94       603

   micro avg       0.95      0.95      0.95      1349
   macro avg       0.95      0.95      0.95      1349
weighted avg       0.95      0.95      0.95      1349



In [115]:
!pip install pandas_ml

    100% |████████████████████████████████| 102kB 3.4MB/s a 0:00:01


In [116]:
from pandas_ml import ConfusionMatrix

ConfusionMatrix(y_test,y_pred)

ValueError: cannot reindex from a duplicate axis

In [ ]:
print(pipe_est_cv.score(X_test,y_test))

In [59]:
clf = LogisticRegression().fit(X_train, y_train)
#print(clf.predict(X))
#print(clf.predict_proba(X)) 
print(clf.score(X_train, y_train))
clf.score(X_test, y_test)

0.5624404194470924


/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5596738324684952

In [ ]:
X_train.columns